In [1]:
import os
import sys
import glob
import pickle
import random

import numpy as np
import pandas as pd
import scipy.stats as stats
from pathlib import Path


sys.path.append('../../')
import src.utils as utils
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler, LabelEncoder

import wandb
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
# from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import EarlyStopping

## config

In [2]:
# config
config = utils.load_config('config.yaml')

# globals variable
SEED = config['globals']['seed']
MAX_EPOCHS = config['globals']['max_epochs']
N_SPLITS = config['globals']['n_splits']
USE_FOLDS = config['globals']['use_folds']
DEBUG = config['globals']['debug']
EXP_MESSAGE = config['globals']['exp_message']
NOTES = config['globals']['notes']
MODEL_SAVE = config['globals']['model_save']
PRETRAINED = config['globals']['pretrained']
PRETRAINED_PATH = config['globals']['pretrained_path']
EXP_NAME = str(Path().resolve()).split('/')[-1]

# seed
utils.set_seed(SEED)

In [3]:
EXP_NAME

'exp006'

In [4]:
!wandb login 1bb2d0449c11d8b987e25c38b9d8dda176310fb6

wandb: Appending key for api.wandb.ai to your netrc file: /home/kuzira/.netrc


## read data

In [5]:
# waypointを補正したdataset
root_dir = Path('../../input/')
with open(root_dir/'kuto_wifi_dataset_v1/train_all.pkl', 'rb') as f:
  train_df = pickle.load( f)

with open(root_dir/'kuto_wifi_dataset_v1/test_all.pkl', 'rb') as f:
  test_df = pickle.load(f)

sub = pd.read_csv(root_dir/'indoor-location-navigation/sample_submission.csv', index_col=0)

In [6]:
train_df.shape

(258125, 210)

In [7]:
train_df['x'].nunique(), train_df['wifi_x'].nunique()

(25336, 235810)

## time_diffの前処理
0~3000sのものはwifi_x,wifi_yを使用  
-1000s~0sのものはもともとのx.yを使用

In [8]:
POSI_DIFF = 4000  # i番目のwaypointを基準に算出したwifi waypointのうち基準のtimestampの直近3sを信頼できるデータとして残す
NEGA_DIFF = -100000  # i番目のwaypointを基準に算出したwifi waypointのうちi+1番目のwaypointに近いものにはi+1のwaypointを座標として与える

train_df.loc[(NEGA_DIFF < train_df['time_diff']) & (train_df['time_diff'] <= 0), 'wifi_x'] = train_df.loc[(NEGA_DIFF< train_df['time_diff']) & (train_df['time_diff'] <= 0), 'x']
train_df.loc[(NEGA_DIFF < train_df['time_diff']) & (train_df['time_diff'] <= 0), 'wifi_y'] = train_df.loc[(NEGA_DIFF < train_df['time_diff']) & (train_df['time_diff'] <= 0), 'y']
train_df = train_df[(NEGA_DIFF < train_df['time_diff']) & (train_df['time_diff'] < POSI_DIFF)].reset_index(drop=True)
train_df

,bssid_0,bssid_1,bssid_2,bssid_3,bssid_4,bssid_5,bssid_6,bssid_7,bssid_8,bssid_9,...,timestamp,x,y,floor,floor_str,path,time_diff,wifi_x,wifi_y,site_id
0,ffe684dfd25a52b046e3108a3f70df46001425f0,97e4a381c3a02ed3151bbf41b8fc1fe5815f5387,5b71ef95e53358c558b78bf3fb152d793729bc8d,2f85d197aec7bfddfee3f53ae9e1b6ed1fc56e92,4328f33869766d0f77a9299441556338e4d8a2b9,df41c761b69993669d4eb875b4474ec44d2372ed,7dc49736770ee9073043134656c89a17529f882f,5a1a7a8496e5f8b88db082de0b412e447e01fd0b,463d0cfe3748eb70524138ed970f03375e8d1030,79179095e63e2b0431e85e3e33b02d95bb135c2e,...,1571625311855,68.064926,241.94000,0,F1,5dad1ca1dc3e2c0006606c3f,1952,67.055846,242.235341,5da958dd46f8266d0737457b
1,97e4a381c3a02ed3151bbf41b8fc1fe5815f5387,5b71ef95e53358c558b78bf3fb152d793729bc8d,cb8f53745c342e2bfd0bf77a5fd8cac6cf303945,ffe684dfd25a52b046e3108a3f70df46001425f0,6b769b9eeb24ff287e6a53736cc7c013d5902901,7dc49736770ee9073043134656c89a17529f882f,3fef087dd272ab07981a60c9cbf6f27460d1364e,5a1a7a8496e5f8b88db082de0b412e447e01fd0b,df41c761b69993669d4eb875b4474ec44d2372ed,d8b1ff62702e02106553be91dc22a0dcf0e780a7,...,1571625311855,68.064926,241.94000,0,F1,5dad1ca1dc3e2c0006606c3f,3900,63.620046,243.155369,5da958dd46f8266d0737457b
2,97e4a381c3a02ed3151bbf41b8fc1fe5815f5387,6b769b9eeb24ff287e6a53736cc7c013d5902901,cb8f53745c342e2bfd0bf77a5fd8cac6cf303945,4328f33869766d0f77a9299441556338e4d8a2b9,3fef087dd272ab07981a60c9cbf6f27460d1364e,5a1a7a8496e5f8b88db082de0b412e447e01fd0b,7dc49736770ee9073043134656c89a17529f882f,df41c761b69993669d4eb875b4474ec44d2372ed,5b71ef95e53358c558b78bf3fb152d793729bc8d,2f85d197aec7bfddfee3f53ae9e1b6ed1fc56e92,...,1571625320099,62.480465,241.71216,0,F1,5dad1ca1dc3e2c0006606c3f,-2385,62.480465,241.712160,5da958dd46f8266d0737457b
3,97e4a381c3a02ed3151bbf41b8fc1fe5815f5387,4328f33869766d0f77a9299441556338e4d8a2b9,3fef087dd272ab07981a60c9cbf6f27460d1364e,5a1a7a8496e5f8b88db082de0b412e447e01fd0b,5d82171d37c5296bcaed8c02745540b491d8a284,471740ef5065943b791f277ada358f9ffc011645,2f85d197aec7bfddfee3f53ae9e1b6ed1fc56e92,afe423c7bc0641d63c95e232ffd65cae3be95351,df41c761b69993669d4eb875b4474ec44d2372ed,5b71ef95e53358c558b78bf3fb152d793729bc8d,...,1571625320099,62.480465,241.71216,0,F1,5dad1ca1dc3e2c0006606c3f,-427,62.480465,241.712160,5da958dd46f8266d0737457b
4,97e4a381c3a02ed3151bbf41b8fc1fe5815f5387,4328f33869766d0f77a9299441556338e4d8a2b9,5a1a7a8496e5f8b88db082de0b412e447e01fd0b,cb8f53745c342e2bfd0bf77a5fd8cac6cf303945,6b769b9eeb24ff287e6a53736cc7c013d5902901,5d82171d37c5296bcaed8c02745540b491d8a284,5b71ef95e53358c558b78bf3fb152d793729bc8d,df41c761b69993669d4eb875b4474ec44d2372ed,2f85d197aec7bfddfee3f53ae9e1b6ed1fc56e92,afe423c7bc0641d63c95e232ffd65cae3be95351,...,1571625320099,62.480465,241.71216,0,F1,5dad1ca1dc3e2c0006606c3f,1528,61.960799,241.710554,5da958dd46f8266d0737457b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228691,993a56b32432fb19bfb4461a0e1a2ead9bcf192f,773cde25cb4e9fd90b11603abd5bf84d83b340e6,19647ac7bb55a673554aa08cafb3b096aac7f32c,a4a410696cb935d542d62afd8e8090dbbc341a16,fc27c0656fc13157bb2f58543d51e8ee972fdf66,c7d8359344120911f8550487f282c241d93c4750,26b22cce3b7694a7d765d9cf329b9065f3fb3a3c,827530050f580378b7aa53fb292dfb8a12b775e1,d64eeb8d997e8d87203479556bbb9efaf7e487fd,22a52f1717436ee378dc44b6d707a3816a65b5e4,...,1573822164854,12.662716,100.47756,1,F2,5dce9eea5516ad00065f04a7,447,12.053943,100.515005,5d27099f03f801723c32511d
228692,773cde25cb4e9fd90b11603abd5bf84d83b340e6,993a56b32432fb19bfb4461a0e1a2ead9bcf192f,19647ac7bb55a673554aa08cafb3b096aac7f32c,fc27c0656fc13157bb2f58543d51e8ee972fdf66,a4a410696cb935d542d62afd8e8090dbbc341a16,827530050f580378b7aa53fb292dfb8a12b775e1,c7d8359344120911f8550487f282c241d93c4750,d64eeb8d997e8d87203479556bbb9efaf7e487fd,b08a1d79d6d4bb2ca71336f7e995c7aa1342aa1f,26b22cce3b7694a7d765d9cf329b9065f3fb3a3c,...,1573822164854,12.662716,100.47756,1,F2,5dce9eea5516ad00065f04a7,2393,10.895453,101.396328,5d27099f03f801723c32511d
228693,993a56b32432fb19bfb4461a0e

BSSIDとRSSIは100ずつ存在しているけど全てが必要なわけではないみたい  
ここでは20だけ取り出している。

In [9]:
# training target features
NUM_FEATS = 80
BSSID_FEATS = [f'bssid_{i}' for i in range(NUM_FEATS)]
RSSI_FEATS  = [f'rssi_{i}' for i in range(NUM_FEATS)]

bssid_NはN個目のBSSIDを示しておりRSSI値が大きい順に番号が振られている。
100個しかない


In [10]:
# get numbers of bssids to embed them in a layer

# train
wifi_bssids = []
# bssidを列ごとにリストに入れていく
for i in range(100):
    wifi_bssids.extend(train_df.iloc[:,i].values.tolist())
wifi_bssids = list(set(wifi_bssids))

train_wifi_bssids_size = len(wifi_bssids)
print(f'BSSID TYPES(train): {train_wifi_bssids_size}')

# test
wifi_bssids_test = []
for i in range(100):
    wifi_bssids_test.extend(test_df.iloc[:,i].values.tolist())
wifi_bssids_test = list(set(wifi_bssids_test))

test_wifi_bssids_size = len(wifi_bssids_test)
print(f'BSSID TYPES(test): {test_wifi_bssids_size}')


wifi_bssids.extend(wifi_bssids_test)
wifi_bssids = list(set(wifi_bssids))
wifi_bssids_size = len(wifi_bssids)
print(f'BSSID TYPES(all): {wifi_bssids_size}')


BSSID TYPES(train): 41263
BSSID TYPES(test): 28592
BSSID TYPES(all): 41277


## preprocessing

In [11]:
# preprocess

le = LabelEncoder()
le.fit(wifi_bssids)
le_site = LabelEncoder()
le_site.fit(train_df['site_id'])

ss = StandardScaler()
ss.fit(train_df.loc[:,RSSI_FEATS])


def preprocess(input_df, le=le, le_site=le_site, ss=ss):
    output_df = input_df.copy()
    # RSSIの正規化
    output_df.loc[:,RSSI_FEATS] = ss.transform(input_df.loc[:,RSSI_FEATS])

    # BSSIDのLE(1からふる)
    for i in BSSID_FEATS:
        output_df.loc[:,i] = le.transform(input_df.loc[:,i])
        output_df.loc[:,i] = output_df.loc[:,i] + 1  # 0からではなく1から番号を振りたいため なぜ？ embeddingのpadding用のダミー変数？

    # site_idのLE
    output_df.loc[:, 'site_id'] = le_site.transform(input_df.loc[:, 'site_id'])

    # なぜ２重でやる？
    # output_df.loc[:,RSSI_FEATS] = ss.transform(output_df.loc[:,RSSI_FEATS])
    return output_df

train = preprocess(train_df)
test = preprocess(test_df)

    

In [12]:
site_count = len(train['site_id'].unique())
site_count

24

In [13]:
train

,bssid_0,bssid_1,bssid_2,bssid_3,bssid_4,bssid_5,bssid_6,bssid_7,bssid_8,bssid_9,...,timestamp,x,y,floor,floor_str,path,time_diff,wifi_x,wifi_y,site_id
0,41259,24542,14848,7716,10988,35935,20286,14608,11474,19548,...,1571625311855,68.064926,241.94000,0,F1,5dad1ca1dc3e2c0006606c3f,1952,67.055846,242.235341,21
1,24542,14848,32846,41259,17436,20286,10464,14608,35935,34935,...,1571625311855,68.064926,241.94000,0,F1,5dad1ca1dc3e2c0006606c3f,3900,63.620046,243.155369,21
2,24542,17436,32846,10988,10464,14608,20286,35935,14848,7716,...,1571625320099,62.480465,241.71216,0,F1,5dad1ca1dc3e2c0006606c3f,-2385,62.480465,241.712160,21
3,24542,10988,10464,14608,15167,11611,7716,28375,35935,14848,...,1571625320099,62.480465,241.71216,0,F1,5dad1ca1dc3e2c0006606c3f,-427,62.480465,241.712160,21
4,24542,10988,14608,32846,17436,15167,14848,35935,7716,28375,...,1571625320099,62.480465,241.71216,0,F1,5dad1ca1dc3e2c0006606c3f,1528,61.960799,241.710554,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228691,24747,19255,4142,26529,40613,32256,6218,21026,34574,5611,...,1573822164854,12.662716,100.47756,1,F2,5dce9eea5516ad00065f04a7,447,12.053943,100.515005,5
228692,19255,24747,4142,40613,26529,21026,32256,34574,28479,6218,...,1573822164854,12.662716,100.47756,1,F2,5dce9eea5516ad00065f04a7,2393,10.895453,101.396328,5
228693,24747,19255,32256,40613,26529,21026,34574,4008,5611,9850,...,1573822173051,7.799886,107.13921,1,F2,5dce9eea5516ad00065f04a7,-3876,7.799886,107.139210,5
228694,24747,32256,40613,19255,26529,1031,28479,9850,4142,33217,...,1573822173051,7.799886,107.13921,1,F2,5dce9eea5516ad00065f04a7,-1946,7.799886,107.139210,5


## PyTorch model
- embedding layerが重要  

In [14]:
# dataset
from torch.utils.data import Dataset, DataLoader
class IndoorDataset(Dataset):
    def __init__(self, df, phase='train'):
        self.df = df
        self.phase = phase
        self.bssid_feats = df[BSSID_FEATS].values.astype(int)
        self.rssi_feats = df[RSSI_FEATS].values.astype(np.float32)
        self.site_id = df['site_id'].values.astype(int)

        if phase in ['train', 'valid']:
            # self.xy = df[['x', 'y']].values.astype(np.float32)
            self.xy = df[['wifi_x', 'wifi_y']].values.astype(np.float32)  # wifiにより補正したx,yを使用
            self.floor = df['floor'].values.astype(np.float32)
        
    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        
        feature = {
            'BSSID_FEATS':self.bssid_feats[idx],
            'RSSI_FEATS':self.rssi_feats[idx],
            'site_id':self.site_id[idx]
        }
        if self.phase in ['train', 'valid']:
            target = {
                'xy':self.xy[idx],
                'floor':self.floor[idx]
            }
        else:
            target = {}
        return feature, target

In [15]:
import torch
from torch import nn

class LSTMModel(nn.Module):
    def __init__(self, wifi_bssids_size, site_count=24, embedding_dim=64):
        super(LSTMModel, self).__init__()
        # bssid
        # ->64次元に圧縮後sequence化にする
        # wifi_bssids_sizeが辞書の数を表す
        self.embedding_layer1 = nn.Sequential(
            nn.Embedding(wifi_bssids_size, embedding_dim),
            nn.Flatten(start_dim=-2)            
        )
        # site
        # ->2次元に圧縮後sequence化する
        # site_countが辞書の数を表す
        self.embedding_layer2 = nn.Sequential(
            nn.Embedding(site_count, 2),
            nn.Flatten(start_dim=-1)           
        )

        # rssi
        # 次元を64倍に線形変換
        self.linear_layer1 = nn.Sequential(
            nn.BatchNorm1d(NUM_FEATS),
            nn.Linear(NUM_FEATS, NUM_FEATS * embedding_dim),
            nn.ReLU()
        )
        
        # bssid, site, rssiの出力size
        feature_size = 2 + (2 * NUM_FEATS * embedding_dim)
        self.linear_layer2 = nn.Sequential(
            nn.BatchNorm1d(feature_size),
            nn.Dropout(0.3),
            nn.Linear(feature_size, 256),
            nn.ReLU()
        )

        self.batch_norm1 = nn.BatchNorm1d(1)
        self.lstm1 = nn.LSTM(input_size=256,hidden_size=128,dropout=0.3, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=128,hidden_size=16,dropout=0.1, batch_first=True)

        self.fc_xy = nn.Linear(16, 2)
        # self.fc_x = nn.Linear(16, 1)
        # self.fc_y = nn.Linear(16, 1)
        self.fc_floor = nn.Linear(16, 1)

    
    def forward(self, x):
        # input embedding
        batch_size = x["site_id"].shape[0]
        x_bssid = self.embedding_layer1(x['BSSID_FEATS'])
        x_site_id = self.embedding_layer2(x['site_id'])
        x_rssi = self.linear_layer1(x['RSSI_FEATS'])
        x = torch.cat([x_bssid, x_site_id, x_rssi], dim=-1)
        x = self.linear_layer2(x)

        # lstm layer
        x = x.view(batch_size, 1, -1)  # [batch, 1]->[batch, 1, 1]
        x = self.batch_norm1(x)
        x, _ = self.lstm1(x)
        x = torch.relu(x)
        x, _ = self.lstm2(x)
        x = torch.relu(x)

        # output [batch, 1, 1] -> [batch]
        # x_ = self.fc_x(x).view(-1)
        # y_ = self.fc_y(x).view(-1)
        xy = self.fc_xy(x).squeeze(1)
        floor = torch.relu(self.fc_floor(x)).view(-1)
        # return {"x":x_, "y":y_, "floor":floor} 
        return {"xy": xy, "floor": floor}

In [16]:
def mean_position_error(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]

def to_np(input):
    return input.detach().cpu().numpy()

In [17]:
def get_optimizer(model: nn.Module, config: dict):
    optimizer_config = config["optimizer"]
    optimizer_name = optimizer_config.get("name")
    base_optimizer_name = optimizer_config.get("base_name")
    optimizer_params = optimizer_config['params']

    if hasattr(optim, optimizer_name):
        optimizer = optim.__getattribute__(optimizer_name)(model.parameters(), **optimizer_params)
        return optimizer
    else:
        base_optimizer = optim.__getattribute__(base_optimizer_name)
        optimizer = globals().get(optimizer_name)(
            model.parameters(), 
            base_optimizer,
            **optimizer_config["params"])
        return  optimizer

def get_scheduler(optimizer, config: dict):
    scheduler_config = config["scheduler"]
    scheduler_name = scheduler_config.get("name")

    if scheduler_name is None:
        return
    else:
        return optim.lr_scheduler.__getattribute__(scheduler_name)(
            optimizer, **scheduler_config["params"])


def get_criterion(config: dict):
    loss_config = config["loss"]
    loss_name = loss_config["name"]
    loss_params = {} if loss_config.get("params") is None else loss_config.get("params")
    if hasattr(nn, loss_name):
        criterion = nn.__getattribute__(loss_name)(**loss_params)
    else:
        criterion = globals().get(loss_name)(**loss_params)

    return criterion

def worker_init_fn(worker_id):                                                          
    np.random.seed(np.random.get_state()[1][0] + worker_id)

In [18]:
# Learner class(pytorch-lighting)
class Learner(pl.LightningModule):
    def __init__(self, model, config):
        super().__init__()
        self.model = model
        self.config = config
        self.xy_criterion = get_criterion(config)
        self.f_criterion = get_criterion(config)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        output = self.model(x)
        loss = self.xy_criterion(output["xy"], y["xy"])
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        output = self.model(x)
        xy_loss = self.xy_criterion(output["xy"], y["xy"])
        f_loss = self.f_criterion(output["floor"], y["floor"])
        loss = xy_loss  # + f_loss
        mpe = mean_position_error(
            to_np(output['xy'][:, 0]), to_np(output['xy'][:, 1]), 0, 
            to_np(y['xy'][:, 0]), to_np(y['xy'][:, 1]), 0)
        
        # floor lossは現状は無視して良い
        self.log(f'Loss/val', loss, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        self.log(f'Loss/xy', xy_loss, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        self.log(f'Loss/floor', f_loss, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        self.log(f'MPE/val', mpe, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = get_optimizer(self.model, self.config)
        scheduler = get_scheduler(optimizer, self.config)
        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "Loss/val"}

In [19]:
# oof
def evaluate(model, loaders, phase):
    x_list = []
    y_list = []
    f_list = []
    with torch.no_grad():
        for batch in loaders[phase]:
            x, y = batch
            output = model(x)
            x_list.append(to_np(output['xy'][:, 0]))
            y_list.append(to_np(output['xy'][:, 1]))
            f_list.append(to_np(output['floor']))

    x_list = np.concatenate(x_list)
    y_list = np.concatenate(y_list)
    f_list = np.concatenate(f_list)
    return x_list, y_list, f_list

## train

In [20]:
oofs = []  # 全てのoofをdfで格納する
predictions = []  # 全ての予測値をdfで格納する
val_scores = []
# skf = model_selection.StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
gkf = model_selection.GroupKFold(n_splits=N_SPLITS)
# 今回はtargetを均等に分ける必要はなくpathが均等に分かれればいいのでskf.split()にpathを与えている。
for fold, (trn_idx, val_idx) in enumerate(gkf.split(train.loc[:, 'path'], groups=train.loc[:, 'path'])):
    # 指定したfoldのみループを回す
    if fold not in USE_FOLDS:
        continue

    print('=' * 20)
    print(f'Fold {fold}')
    print('=' * 20)

    # train/valid data
    trn_df = train.loc[trn_idx, BSSID_FEATS + RSSI_FEATS + ['site_id', 'wifi_x','wifi_y','floor']].reset_index(drop=True)
    val_df = train.loc[val_idx, BSSID_FEATS + RSSI_FEATS + ['site_id', 'wifi_x','wifi_y','floor']].reset_index(drop=True)

    # data loader
    loaders = {}
    loader_config = config["loader"]
    loaders["train"] = DataLoader(IndoorDataset(trn_df, phase="train"), **loader_config["train"], worker_init_fn=worker_init_fn) 
    loaders["valid"] = DataLoader(IndoorDataset(val_df, phase="valid"), **loader_config["valid"], worker_init_fn=worker_init_fn)
    loaders["test"] = DataLoader(IndoorDataset(test, phase="test"), **loader_config["test"], worker_init_fn=worker_init_fn)

    # model
    model = LSTMModel(wifi_bssids_size+1, site_count)  # +1としているのはLEを1スタートで始めているため
    model_name = model.__class__.__name__

    # callbacks
    callbacks = []
    checkpoint_callback = ModelCheckpoint(
        monitor=f'Loss/val',
        mode='min',
        dirpath="./",
        verbose=False,
        filename=f'{model_name}-{fold}')
    
    if MODEL_SAVE:
        callbacks.append(checkpoint_callback)

    early_stop_callback = EarlyStopping(
        monitor='Loss/val',
        min_delta=0.00,
        patience=5,
        verbose=False,
        mode='min')
    # callbacks.append(early_stop_callback)

    # loggers
    RUN_NAME = EXP_NAME + "_" + EXP_MESSAGE
    wandb.init(project='indoor', notes=NOTES, entity='kuto5046', group=RUN_NAME)
    wandb.run.name = RUN_NAME + f'-fold-{fold}'
    wandb_config = wandb.config
    wandb_config.model_name = model_name
    wandb_config.LB = None
    wandb.watch(model)
    
    
    loggers = []
    loggers.append(WandbLogger())

    learner = Learner(model, config)
    # pretrained flag
    if PRETRAINED:
        ckpt = torch.load(PRETRAINED_PATH + f'{model_name}-{fold}.ckpt')
        learner.load_state_dict(ckpt['state_dict'])

    trainer = pl.Trainer(
        logger=loggers, 
        callbacks=callbacks,
        max_epochs=MAX_EPOCHS,
        gpus=[0],
        fast_dev_run=DEBUG,
        deterministic=True,
        precision=16,
        progress_bar_refresh_rate=0  # vscodeの時progress barの動作が遅いので表示しない
        )

    trainer.fit(learner, train_dataloader=loaders['train'], val_dataloaders=loaders['valid'])

    #############
    # validation (to make oof)
    #############
    model.eval()
    oof_x, oof_y, oof_f = evaluate(model, loaders, phase="valid")
    val_df["oof_x"] = oof_x
    val_df["oof_y"] = oof_y
    val_df["oof_floor"] = oof_f
    oofs.append(val_df)
    
    val_score = mean_position_error(
        val_df["oof_x"].values, val_df["oof_y"].values, 0,
        val_df['wifi_x'].values, val_df['wifi_y'].values, 0)
    val_scores.append(val_score)
    print(f"fold {fold}: mean position error {val_score}")

    #############
    # inference
    #############n

    preds_x, preds_y, preds_f = evaluate(model, loaders, phase="test")
    test_preds = pd.DataFrame(np.stack((preds_f, preds_x, preds_y))).T
    test_preds.columns = sub.columns
    test_preds["site_path_timestamp"] = test["site_path_timestamp"]
    test_preds["floor"] = test_preds["floor"].astype(int)
    test_preds.to_csv(f'{EXP_NAME}_fold{fold}.csv', index=False)
    predictions.append(test_preds)
    wandb.finish()

Fold 0
/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/rnn.py:58: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/rnn.py:58: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kuto5046 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.23 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.

  | Name         | Type      | Params
-------------------------------------------
0 | model        | LSTMModel | 5.9 M 
1 | xy_criterion | MSELoss   | 0     
2 | f_criterion  | MSELoss   | 0     
-------------------------------------------
5.9 M     Trainable params
0         Non-trainable params
5.9 M     Total params
23.626    Total estimated model params size (MB)
fold 0: mean position error 8.533997512667785


Loss/val,57.4697
Loss/xy,57.4697
Loss/floor,4.71926
MPE/val,8.53438
epoch,299
trainer/global_step,107399
_runtime,1237
_timestamp,1617027342
_step,299


Loss/val,█▇▅▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss/xy,█▇▅▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss/floor,▅▅▅▅▅▅▃▁▃▄▅▆▆▆▇▇▇▇▇▇██████████████▇▇▇▇▇▇
MPE/val,█▇▆▅▅▅▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


Fold 1
wandb: wandb version 0.10.23 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


...

Loss/val,█▇▆▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss/xy,█▇▆▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss/floor,██████▇▃▅▅▁▁▁▁▁▁▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
MPE/val,█▇▆▅▅▅▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


Fold 4
wandb: wandb version 0.10.23 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.

  | Name         | Type      | Params
-------------------------------------------
0 | model        | LSTMModel | 5.9 M 
1 | xy_criterion | MSELoss   | 0     
2 | f_criterion  | MSELoss   | 0     
-------------------------------------------
5.9 M     Trainable params
0         Non-trainable params
5.9 M     Total params
23.626    Total estimated model params size (MB)
fold 4: mean position error 9.150369947871221


Loss/val,76.39194
Loss/xy,76.39194
Loss/floor,5.24786
MPE/val,9.14999
epoch,299
trainer/global_step,107399
_runtime,1287
_timestamp,1617032447
_step,299


Loss/val,█▇▆▅▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss/xy,█▇▆▅▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss/floor,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
MPE/val,█▇▆▆▅▅▄▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


In [21]:
if len(USE_FOLDS) > 1:
    oofs_df = pd.concat(oofs)
else:
    oofs_df = oofs[0]
oofs_df.to_csv("oof.csv", index=False)
oofs_df

,bssid_0,bssid_1,bssid_2,bssid_3,bssid_4,bssid_5,bssid_6,bssid_7,bssid_8,bssid_9,...,rssi_77,rssi_78,rssi_79,site_id,wifi_x,wifi_y,floor,oof_x,oof_y,oof_floor
0,17692,37534,24386,5761,24542,28375,35935,8141,7716,15167,...,0.544768,0.551930,0.559533,21,69.418923,221.827425,0,67.146973,220.738937,0.557049
1,17692,5761,24386,3939,37534,15167,11611,7716,25455,28375,...,0.544768,0.549360,0.556982,21,69.692130,224.579247,0,66.856201,219.382462,0.565972
2,8416,25455,28061,3939,17692,15167,14608,11611,10988,41259,...,0.539595,0.546791,0.554430,21,68.511270,226.272460,0,66.766937,223.298981,0.566208
3,24542,7716,11611,35935,15167,11474,28375,14848,10988,41259,...,0.539595,0.546791,0.554430,21,68.511270,226.272460,0,65.731834,231.712677,0.544176
4,10464,28375,41259,7716,17692,11474,35935,14848,15167,8416,...,0.539595,0.546791,0.554430,21,68.637783,228.061851,0,65.583000,232.781006,0.548793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45734,2511,34574,12078,35822,3485,24834,23400,17548,14623,17899,...,0.537008,0.544222,0.551879,5,15.299171,14.961337,1,34.292912,29.164856,0.000000
45735,2511,34574,3485,35822,24834,23400,17548,17899,14623,5686,...,0.539595,0.546791,0.551879,5,17.330584,13.464786,1,34.922638,28.536509,0.000000
45736,2511,34574,3485,35822,24834,23400,17899,14623,16696,5686,...,0.542181,0.549360,0.556982,5,21.582791,13.336142,1,33.454681,30.001345,0.000000
45737,2511,34574,3485,16233,14623,5686,433,16696,7255,17899,...,0.542181,0.549360,0.554430,5,25.071220,14.849329,1,32.069366,31.383858,0.000000


In [22]:
if len(USE_FOLDS) > 1:
    # foldの結果を平均した後、reindexでsubmission fileにindexを合わせる
    all_preds = pd.concat(predictions).groupby('site_path_timestamp').mean().reindex(sub.index)
else:
    all_preds = predictions[0].reindex(sub.index)
all_preds

,floor,x,y
site_path_timestamp,,,
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,0,90.571655,103.244865
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000009017,0,86.755478,100.798737
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000015326,0,85.054337,105.064995
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000018763,0,89.280891,108.794083
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000022328,0,87.927269,108.296745
...,...,...,...
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000082589,0,213.258698,93.442741
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000085758,0,211.048828,99.856056
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000090895,0,207.109894,106.287132


In [23]:
# floorの数値を置換
simple_accurate_99 = pd.read_csv(root_dir / 'simple-99-accurate-floor-model/submission.csv')
all_preds['floor'] = simple_accurate_99['floor'].values
all_preds

,floor,x,y
site_path_timestamp,,,
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,0,90.571655,103.244865
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000009017,0,86.755478,100.798737
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000015326,0,85.054337,105.064995
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000018763,0,89.280891,108.794083
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000022328,0,87.927269,108.296745
...,...,...,...
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000082589,5,213.258698,93.442741
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000085758,5,211.048828,99.856056
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000090895,5,207.109894,106.287132


In [24]:
all_preds.to_csv(RUN_NAME + '.csv')

In [25]:
print(f"CV:{np.mean(val_scores)}")

CV:8.902057890949
